In [1]:
from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics

from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


#  Importing Tensorflow
import tensorflow as tf
# Importing keras
from tensorflow import keras
# Importing Tuner
import keras_tuner as kt

print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')




Automatically created module for IPython interactive environment


In [2]:
# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

In [3]:


# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples) # Number of Samples
print("n_features: %d" % n_features) # Number of Features
print("n_classes: %d" % n_classes) # Number of Classes

# Dimensions of images
print("Dimensions:")
print("Picture height: %d" % h)
print("Picture width: %d" % w)

# Names of targets
target_names

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7
Dimensions:
Picture height: 50
Picture width: 37


array(['Ariel Sharon', 'Colin Powell', 'Donald Rumsfeld', 'George W Bush',
       'Gerhard Schroeder', 'Hugo Chavez', 'Tony Blair'], dtype='<U17')

In [4]:
# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)


# Random state is used to make the randomization constant between runs

In [5]:
# Scale images to the [0, 1] range
# X_train = X_train / 255.0
# X_test = X_test / 255.0

# Make sure images have shape (50, 37, 1), based on image dimensions 
X_train = X_train.reshape(len(X_train), 50, 37, 1)
X_test = X_test.reshape(len(X_test), 50, 37, 1)


In [6]:
####################################################################################
# Hyper Parameter

def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(50, 37, 1)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

2022-10-06 14:34:58.060849: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.6164383292198181

Best val_accuracy So Far: 0.6164383292198181
Total elapsed time: 00h 00m 42s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
28/28 [==============================] - 1s 10ms/step - loss: 432.1040 - accuracy: 0.2457 - val_loss: 115.8629 - val_accuracy: 0.3607
Epoch 2/50
28/28 [==============================] - 0s 5ms/step - loss: 39.8650 - accuracy: 0.3589 - val_loss: 31.9568 - val_accuracy: 0.3470
Epoch 3/50
28/28 [==============================] - 0s 5ms/step - loss: 41.4019 - accuracy: 0.3497 - val_loss: 29.3963 - val_accuracy: 0.4201
Epoch 4/50
28/28 [==============================] - 0s 5ms/step - loss: 25.6794 - accuracy: 0.4126 - val_loss: 16.3852 - val_accuracy: 0.4429
Epoch 5/50
28/28 [==============================] - 0s 5ms/step - loss: 13.0639 - accuracy: 0.4903 - val_loss: 17.3434 - val_accuracy: 0.4521
Epoch 6/50
28/28 [==============================] - 0s 5ms/step - loss: 11.7211 - accuracy: 0.5303 - val_loss: 18.7622 - val_accuracy: 0.4566
Epoch 7/50
28/28 [==============================] - 0s 5ms/step - loss: 10.4648 - accuracy: 0.5406 - val_loss: 12.3500 - val_accuracy: 0.6073
Epo

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/36
28/28 [==============================] - 1s 10ms/step - loss: 372.1999 - accuracy: 0.2583 - val_loss: 105.2199 - val_accuracy: 0.0502
Epoch 2/36
28/28 [==============================] - 0s 5ms/step - loss: 89.9509 - accuracy: 0.3429 - val_loss: 31.1878 - val_accuracy: 0.4521
Epoch 3/36
28/28 [==============================] - 0s 5ms/step - loss: 27.1392 - accuracy: 0.4880 - val_loss: 31.7199 - val_accuracy: 0.3014
Epoch 4/36
28/28 [==============================] - 0s 5ms/step - loss: 22.8641 - accuracy: 0.5086 - val_loss: 33.9327 - val_accuracy: 0.5799
Epoch 5/36
28/28 [==============================] - 0s 5ms/step - loss: 27.1925 - accuracy: 0.5029 - val_loss: 29.9053 - val_accuracy: 0.4521
Epoch 6/36
28/28 [==============================] - 0s 5ms/step - loss: 31.2019 - accuracy: 0.4971 - val_loss: 52.3817 - val_accuracy: 0.4612
Epoch 7/36
28/28 [==============================] - 0s 5ms/step - loss: 27.4588 - accuracy: 0.5280 - val_loss: 23.1168 - val_accuracy: 0.5662
Epo

In [12]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

7/7 [==============================] - 0s 2ms/step - loss: 5.2904 - accuracy: 0.7474
[test loss, test accuracy]: [5.2904205322265625, 0.7474226951599121]


In [13]:
# Code collected and adapted from https://www.tensorflow.org/tutorials/keras/keras_tuner